 # Homework Corrector

 This tool works by Vectorising text and running cosine similarity to compare the level of similarity between students' essays

 Before Checking for Plagiarism between students, this code offers Cleaning the text in the dataframe in case it was scrapped as html documents

 It also offers the ability to delete stop words to better compare the essays

 It has the ability to Highlight wrong semantic and grammar in both English and Arabic

 And lastly we compare all essays together to find similar items helping teachers grade and find plagiarised essays

    Importing Libraries

In [1]:
import pandas as pd

from bs4 import BeautifulSoup

import re

#arabic correction library
from ar_corrector.corrector import Corrector

#english correction library
import language_tool_python  

#Plagiarism Detection Libraries 
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#Stopwords Cleaner Libraries
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


    Reading Scraped Articles CSV File

In [2]:
articles_files = pd.read_csv('./articles.csv')

# EDA

## HTML & Foriegn symbols Cleaner

In [3]:
articles_files.head()

,ArticleId,EncyclopediaId,ArabicTitle,Arabic_title_n_p,EnglishTitle,Intro,Body,ResourceIdCover,CatId,MainCatId,Metadata,IsEditing,dateOfPublication,showed,deleted_at
0,154,12,كتاب البديع لابن المعتز,كتاب البديع لابن المعتز,NaN,<p>كتاب البديع لعبد الله بن المعتزّ (247هـ/861...,"{""paragraphs"":[{""id"":1,""Type"":""intro"",""Title"":...",NaN,425,356,NaN,0,2010-01-01,1,NaN
1,159,12,المَثَل السائر لابن الأثير,المثل السائر لابن الاثير,NaN,<p>كتاب المثل السائر لابن الأَثير الكاتب (558-...,"{""paragraphs"":[{""id"":1,""Type"":""intro"",""Title"":...",NaN,463,356,NaN,0,2010-01-01,1,NaN
2,165,12,أساتذته,ابن المعتز,NaN,<p>هو أبو العبّاس عبد الله بن المعتزّ بن المتو...,"{""paragraphs"":[{""id"":1,""Type"":""intro"",""Title"":...",NaN,426,356,NaN,0,2010-01-01,1,NaN
3,172,12,ابن طَباطَبا (ت 322هـ),ابن طباطبا (ت 322هـ),NaN,<p>هو محمد بن أحمد بن محمد بن أحمد بن إسماعيل ...,"{""paragraphs"":[{""id"":1,""Type"":""intro"",""Title"":...",NaN,434,356,NaN,0,2010-01-01,1,NaN
4,178,12,ابن الأثير,ابن الاثير,NaN,<p>أبو الفتح نصر الله ضياء الدين بن أبي الكرم ...,"{""paragraphs"":[{""id"":1,""Type"":""intro"",""Title"":...",NaN,464,356,NaN,0,2010-01-01,1,NaN


In [4]:
articles_files.drop(columns=['EncyclopediaId', 'Arabic_title_n_p', 'EnglishTitle', 'ResourceIdCover', 'Metadata', 'IsEditing', 'dateOfPublication', 'showed', 'CatId', 'MainCatId', 'deleted_at'], inplace = True)
articles_files.head()

,ArticleId,ArabicTitle,Intro,Body
0,154,كتاب البديع لابن المعتز,<p>كتاب البديع لعبد الله بن المعتزّ (247هـ/861...,"{""paragraphs"":[{""id"":1,""Type"":""intro"",""Title"":..."
1,159,المَثَل السائر لابن الأثير,<p>كتاب المثل السائر لابن الأَثير الكاتب (558-...,"{""paragraphs"":[{""id"":1,""Type"":""intro"",""Title"":..."
2,165,أساتذته,<p>هو أبو العبّاس عبد الله بن المعتزّ بن المتو...,"{""paragraphs"":[{""id"":1,""Type"":""intro"",""Title"":..."
3,172,ابن طَباطَبا (ت 322هـ),<p>هو محمد بن أحمد بن محمد بن أحمد بن إسماعيل ...,"{""paragraphs"":[{""id"":1,""Type"":""intro"",""Title"":..."
4,178,ابن الأثير,<p>أبو الفتح نصر الله ضياء الدين بن أبي الكرم ...,"{""paragraphs"":[{""id"":1,""Type"":""intro"",""Title"":..."


In [5]:
articles_files.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4392 entries, 0 to 4391
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ArticleId    4392 non-null   int64 
 1   ArabicTitle  4392 non-null   object
 2   Intro        4388 non-null   object
 3   Body         4392 non-null   object
dtypes: int64(1), object(3)
memory usage: 137.4+ KB


    Removing HTML Tags and non Arabic symbols

In [6]:

def remove_non_arabic_characters(data_frame: pd.DataFrame, column_name: str):

    data_frame[column_name] = data_frame[column_name].apply(lambda text: re.sub(r'([@A-Za-z0-9_ـــــــــــــ]+)|[^\w\s]|#|http\S+', '', str(text)))


In [7]:
remove_non_arabic_characters(articles_files, 'Body')
remove_non_arabic_characters(articles_files, 'Intro')

In [8]:
articles_files.head()

,ArticleId,ArabicTitle,Intro,Body
0,154,كتاب البديع لابن المعتز,كتاب البديع لعبد الله بن المعتز هم هم ويقع في...,كتاب البديع لابن المعتزكتاب البديع لابن المعتز...
1,159,المَثَل السائر لابن الأثير,كتاب المثل السائر لابن الأثير الكاتب ه م من أه...,المثل السائر لابن الأثيرالمثل السائر لابن الاث...
2,165,أساتذته,هو أبو العباس عبد الله بن المعتز بن المتوكل بن...,أساتذتهابن المعتزهو أبو العباس عبد الله بن الم...
3,172,ابن طَباطَبا (ت 322هـ),هو محمد بن أحمد بن محمد بن أحمد بن إسماعيل بن ...,ابن طباطبا ت هابن طباطبا ت ههو محمد بن أحمد بن...
4,178,ابن الأثير,أبو الفتح نصر الله ضياء الدين بن أبي الكرم محم...,ابن الأثيرابن الاثيرأبو الفتح نصر الله ضياء ال...


In [9]:
# articles_files['Intro'] = articles_files['Intro'].str.replace(r'<[^<>]*>', '', regex=True)

 ## StopWords Cleaner

    Check Arabic Stopwords

In [10]:
print (stopwords.words('arabic') [620:680])

['ثمنمئة', 'تسعمئة', 'مائة', 'ثلاثمائة', 'أربعمائة', 'خمسمائة', 'ستمائة', 'سبعمائة', 'ثمانمئة', 'تسعمائة', 'عشرون', 'ثلاثون', 'اربعون', 'خمسون', 'ستون', 'سبعون', 'ثمانون', 'تسعون', 'عشرين', 'ثلاثين', 'اربعين', 'خمسين', 'ستين', 'سبعين', 'ثمانين', 'تسعين', 'بضع', 'نيف', 'أجمع', 'جميع', 'عامة', 'عين', 'نفس', 'لا سيما', 'أصلا', 'أهلا', 'أيضا', 'بؤسا', 'بعدا', 'بغتة', 'تعسا', 'حقا', 'حمدا', 'خلافا', 'خاصة', 'دواليك', 'سحقا', 'سرا', 'سمعا', 'صبرا', 'صدقا', 'صراحة', 'طرا', 'عجبا', 'عيانا', 'غالبا', 'فرادى', 'فضلا', 'قاطبة', 'كثيرا']


    Define removing stopwords from string function

In [11]:
def remove_arabic_stopwords_from_string(text_string: str):
    
    arabic_stopwords = stopwords.words('arabic')

    words = text_string.split(' ')

    cleaned_words = [word for word in words if word not in arabic_stopwords]

    clean_text = ' '.join(cleaned_words)

    return clean_text

In [12]:
corpus = remove_arabic_stopwords_from_string('ذهب الولد الى التفاحة و هو جائع و صغير ثلاثين خاصة')

print(corpus)

الولد الى التفاحة جائع صغير


In [13]:
# from nltk.corpus import stopwords
# stop = stopwords.words('arabic')

# test = articles_files

# test['Body'] = test['Body'].astype('string', errors='ignore')

# test['Body_without_stopwords'] = test['Body'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))


# test.head()

In [14]:
def remove_arabic_stopwords_from_datafram(data_frame: pd.DataFrame, column_name: str, cleaned_column_name:str):

    arabic_stopwords = stopwords.words('arabic')
    
    test = data_frame

    test[column_name] = test[column_name].astype('string', errors='ignore')

    test[cleaned_column_name] = test[column_name].apply(lambda x: ' '.join([word for word in x.split() if word not in (arabic_stopwords)]))

In [15]:
remove_arabic_stopwords_from_datafram(articles_files, 'Body', 'cleaned_body')
remove_arabic_stopwords_from_datafram(articles_files, 'Intro', 'cleaned_intro')

In [16]:
articles_files.head()

,ArticleId,ArabicTitle,Intro,Body,cleaned_body,cleaned_intro
0,154,كتاب البديع لابن المعتز,كتاب البديع لعبد الله بن المعتز هم هم ويقع في...,كتاب البديع لابن المعتزكتاب البديع لابن المعتز...,كتاب البديع لابن المعتزكتاب البديع لابن المعتز...,كتاب البديع لعبد الله بن المعتز ويقع صفحة مذيل...
1,159,المَثَل السائر لابن الأثير,كتاب المثل السائر لابن الأثير الكاتب ه م من أه...,المثل السائر لابن الأثيرالمثل السائر لابن الاث...,المثل السائر لابن الأثيرالمثل السائر لابن الاث...,كتاب المثل السائر لابن الأثير الكاتب أهم المؤل...
2,165,أساتذته,هو أبو العباس عبد الله بن المعتز بن المتوكل بن...,أساتذتهابن المعتزهو أبو العباس عبد الله بن الم...,أساتذتهابن المعتزهو العباس عبد الله بن المعتز ...,العباس عبد الله بن المعتز بن المتوكل بن المعتص...
3,172,ابن طَباطَبا (ت 322هـ),هو محمد بن أحمد بن محمد بن أحمد بن إسماعيل بن ...,ابن طباطبا ت هابن طباطبا ت ههو محمد بن أحمد بن...,ابن طباطبا هابن طباطبا ههو محمد بن أحمد بن محم...,محمد بن أحمد بن محمد بن أحمد بن إسماعيل بن إبر...
4,178,ابن الأثير,أبو الفتح نصر الله ضياء الدين بن أبي الكرم محم...,ابن الأثيرابن الاثيرأبو الفتح نصر الله ضياء ال...,ابن الأثيرابن الاثيرأبو الفتح نصر الله ضياء ال...,الفتح نصر الله ضياء الدين بن أبي الكرم محمد بن...


    Saving Clean Data in new CSV File

In [17]:
#articles_files_cleaned = articles_files.to_csv('/home/elearning/Desktop/Artificial_Intelligence_Code/Plagiarism/articles_cleaned.csv')

# Syntax & Semantic Correction

## Arabic Corrector

    Defining Corrector and trying library

In [18]:
corr = Corrector()

sent = ' قواءص  التمذد في تشاد أنها تواضضل طريقها للعاحمة'
print(corr.contextual_correct(sent)) 

sent = 'اتتنتهى حدث آبل المنتظو بالإعلاخ عن مموعة من المنتجات'
print(corr.contextual_correct(sent))

قراءة التمرد في تشاد أنها تواصل طريقها للعاصمة
انتهى حدث آبل المنتظر الإعلان عن مجموعة من المنتجات


    Defining Corrector Function for Dataframe

In [19]:
def arabic_corrector_for_dataframe(data_frame: pd.DataFrame, column_name: str):

    data_frame[column_name] = data_frame[column_name].apply(lambda text: corr.contextual_correct(text))

#arabic_corrector_for_dataframe(articles_files, 'Intro')

    Defining Corrector Function for Text

In [20]:
def arabic_corrector_for_text(text):
    
    print(corr.contextual_correct(text))

## English Corrector

### Version 1

    Entering text example

In [21]:
my_tool = language_tool_python.LanguageTool('en-US')  
my_text = """LanguageTool provides utility to check grammar and spelling errors. We just have to paste the text here and click the 'Check Text' button. Click the colored phrases for for information on potential errors. or we can use this text too see an some of the issues that LanguageTool can dedect. Whot do someone thinks of grammar checkers? Please not that they are not perfect. Style problems get a blue marker: It is 7 P.M. in the evening. The weather was nice on Monday, 22 November 2021"""   

    Defining Corrector Function for Text

In [22]:
def english_text_corrector(tool, text):
    
    matches = tool.check(text)

    #empty lists
    Mistakes = [] 
    Corrections = []  
    StartPositions = []  
    EndPositions = []  

    for rules in matches:
        if len(rules.replacements) > 0:  
            StartPositions.append(rules.offset)  
            EndPositions.append(rules.errorLength + rules.offset)  
            Mistakes.append(my_text[rules.offset : rules.errorLength + rules.offset])  
            Corrections.append(rules.replacements[0]) 

    print("Mistakes made")
    print (Mistakes)
    print ("\nRecommended Corrections")
    print(Corrections)
    print ("\nMistake Starting character number")
    print(StartPositions)
    print ("\nMistake EndPoint character number")
    print(EndPositions)

    mistakes_number = len (Mistakes)

    print( "\nNumber of mistakes made " + str(mistakes_number))
    #return mistakes_number

    Printing Corrector Results

In [24]:
english_text_corrector(my_tool, my_text)

Mistakes made
['for for', 'or', 'too see', 'an', 'dedect', 'Whot', 'not', 'P.M. in the evening']

Recommended Corrections
['for', 'Or', 'to see', 'a', 'detect', 'Who', 'note', 'P.M.']

Mistake Starting character number
[165, 206, 230, 238, 282, 290, 341, 414]

Mistake EndPoint character number
[172, 208, 237, 240, 288, 294, 344, 433]

Number of mistakes made 8


# Plagiarism Detection

This tool works by Vectorising text and running cosine similarity to compare the level of similarity between students' essays

In [25]:
path = './docs'
student_files = [os.path.join(path, doc) for doc in os.listdir(path) if
                 doc.endswith('.txt')]

In [26]:
student_notes = [open(File).read() for File in student_files]

vectorize = lambda text: TfidfVectorizer().fit_transform(text).toarray()
similarity = lambda doc1, doc2: cosine_similarity([doc1, doc2])

vectors = vectorize(student_notes)
s_vectors = list(zip(student_files, vectors))

In [27]:
# def check_plagiarism():

#     plagiarism_results = set()
#     global s_vectors

#     for student_a, text_vector_a in s_vectors:
        
#         new_vectors = s_vectors.copy()
#         current_index = new_vectors.index((student_a, text_vector_a))
#         del new_vectors[current_index]

#         for student_b, text_vector_b in new_vectors:

#             sim_score = similarity(text_vector_a, text_vector_b)[0][1]
#             student_pair = sorted((student_a, student_b))
#             score = (student_pair[0].replace(path, ''), student_pair[1].replace(path, ''), sim_score)
#             plagiarism_results.add(score)

#     return plagiarism_results

# for data in check_plagiarism():
#     print(data)


In [28]:
import pysimilar
from pysimilar import compare

def check_plagiarism1():

    plagiarism_results = set()
    global s_vectors

    for student_a, text_vector_a in s_vectors:
        
        new_vectors = s_vectors.copy()
        current_index = new_vectors.index((student_a, text_vector_a))
        del new_vectors[current_index]

        for student_b, text_vector_b in new_vectors:

            sim_score = compare(student_a,student_b,isfile=True)
            student_pair = sorted((student_a, student_b))
            score = (student_pair[0].replace(path, ''), student_pair[1].replace(path, ''), sim_score)
            plagiarism_results.add(score)
 
    return plagiarism_results
  
check_plagiarism1()

{('/student1.txt', '/student2.txt', 0.9998461681558491),
 ('/student1.txt', '/student3.txt', 0.9781412960576044),
 ('/student1.txt', '/student4.txt', 0.8545061717430134),
 ('/student1.txt', '/student5.txt', 0.8478847731467942),
 ('/student1.txt', '/student6.txt', 0.8981156438822879),
 ('/student1.txt', '/student6.txt', 0.898115643882288),
 ('/student1.txt', '/student7.txt', 0.7687564712767716),
 ('/student2.txt', '/student3.txt', 0.9772091581862569),
 ('/student2.txt', '/student4.txt', 0.8546828481082853),
 ('/student2.txt', '/student5.txt', 0.8483698350285741),
 ('/student2.txt', '/student6.txt', 0.8978243246628719),
 ('/student2.txt', '/student7.txt', 0.7617673199367186),
 ('/student2.txt', '/student7.txt', 0.7617673199367188),
 ('/student3.txt', '/student4.txt', 0.8497815302431857),
 ('/student3.txt', '/student5.txt', 0.8385451618193595),
 ('/student3.txt', '/student6.txt', 0.9034976415729802),
 ('/student3.txt', '/student6.txt', 0.9034976415729803),
 ('/student3.txt', '/student7.tx